# Phase 4 Playbook Demo

This notebook demonstrates how the Phase 4 LangGraph agents collaborate using the tool wrapping and telemetry features implemented in the MCP layer. It builds a minimal orchestration run with in-memory stubs so you can validate behaviour without external services.

## Prerequisites

* Ensure the backend virtual environment is active (`.venv`).
* Run `uvicorn app.main:app --reload` in a separate terminal if you want to compare the notebook flow with the live API.
* Install optional visualization dependencies (`pip install rich`) if you prefer coloured console output (not required for this walkthrough).

In [ ]:
import asyncio
from dataclasses import dataclass
from typing import Any, Awaitable, Callable

from app.agents.base import AgentContext
from app.agents.enterprise import EnterpriseAgent
from app.agents.research import ResearchAgent
from app.orchestration.graph import Orchestrator
from app.services.tools import ToolInvocationResult


class InMemoryStore:
    def __init__(self) -> None:
        self.records: dict[str, list[Any]] = {}

    async def store_working_memory(self, task_id: str, payload: Any) -> None:
        self.records.setdefault(task_id, []).append({"working": payload})

    async def store_ephemeral_memory(self, task_id: str, payload: dict[str, Any]) -> None:
        self.records.setdefault(task_id, []).append({"ephemeral": payload})


class EchoLLM:
    async def generate(self, prompt: str, *, system_prompt: str | None = None, temperature: float | None = None) -> str:
        prefix = system_prompt or "system"
        return f"{prefix}: {prompt[:120]}"

    async def chat(self, messages: list[Any]) -> str:
        return "Chat flow not required for this demo."


class DummyToolService:
    async def invoke(self, tool: str, payload: dict[str, Any]) -> ToolInvocationResult:
        if tool == "research.search":
            response = {
                "results": [
                    {"summary": "AI adoption can double GTM efficiency", "source": "https://example.com/ai-report"},
                    {"summary": "Enterprise buyers prioritise governance", "source": "https://example.com/governance"},
                ]
            }
            return ToolInvocationResult(
                tool=tool,
                resolved_tool="search/tavily",
                payload=payload,
                response=response,
                cached=False,
                latency=0.012,
            )
        if tool == "enterprise.playbook":
            response = {
                "actions": [
                    {"action": "Launch GTM readiness stand-up", "impact": "Align marketing, sales, and success stakeholders", "origin": "notion"},
                    {"action": "Draft governance FAQ", "impact": "Address buyer risk narratives proactively", "origin": "policy_checker"},
                ]
            }
            return ToolInvocationResult(
                tool=tool,
                resolved_tool="enterprise/playbook",
                payload=payload,
                response=response,
                cached=False,
                latency=0.018,
            )
        raise RuntimeError(f"Unsupported tool alias: {tool}")


async def run_demo(progress_cb: Callable[[dict[str, Any]], Awaitable[None]] | None = None) -> dict[str, Any]:
    orchestrator = Orchestrator(agents=[ResearchAgent(), EnterpriseAgent()])
    memory = InMemoryStore()
    context = AgentContext(
        memory=memory,
        llm=EchoLLM(),
        context=None,
        tools=DummyToolService(),
        scorer=None,
    )

    task_state = {
        "id": "demo-001",
        "prompt": "Summarize GTM guidance for the platform launch and translate it into an executive playbook.",
        "metadata": {"segment": "enterprise", "priority": "high"},
    }

    result = await orchestrator.route_task(task_state, context=context, progress_cb=progress_cb)
    return {"result": result, "memory_records": memory.records}

In [ ]:
async def log_progress(event: dict[str, Any]) -> None:
    print(f"[{event['agent']}] {event['event']} (latency={event.get('latency')}s)")

demo_payload = await run_demo(progress_cb=log_progress)
demo_payload

## Observability Notes

* The notebook uses the same `Orchestrator` and agent implementations that the FastAPI layer consumes.
* Real tool invocations stream structured telemetry via `ToolService.instrument`; in this stubbed run we focus on the agent lifecycle hooks.
* To connect notebook experiments with the live SSE stream, point a browser or CLI client at `/submit_task/stream` and compare the emitted `tool_invocation` events.

Next steps:
1. Swap the dummy services with instances created from `Settings` to run against local Redis/Qdrant/Ollama.
2. Capture example outputs and push them into the docs repository for product demos.
3. Extend the notebook with visualizations (e.g. confidence breakdown bar charts) once Prometheus metrics are exported.